In [1]:
import pandas as pd

In [2]:
cols = pd.read_csv("../Resources/Columns.csv")
data = pd.read_csv("../Resources/nhes_19_pfi_v1_0.csv")

col_list = []

In [3]:
# Load column names to list from CSV
for col, row in cols.iteritems():
    col_list = row.values

col_list

array(['SEGRADES', 'CENREG', 'SCH_TYPE', 'EDCPUB', 'EDCCAT', 'EDCREL',
       'EDCPRI', 'EDCINTK12', 'EDCHSFL', 'DISTASSI', 'SCHRTSCHL',
       'SCHLMAGNET', 'SPBSCH ', 'SOTHRSCH ', 'STUTR ', 'SOTHSCH ',
       'SEENJOY', 'SEABSNT', 'FCSCHOOL', 'FCTEACHR', 'FCSTDS',
       'FCSUPPRT ', 'FHHOME', 'FHWKHRS', 'FOSTORY2X', 'FOCRAFTS',
       'FOGAMES', 'FOBUILDX', 'FOSPORT', 'FORESPON', 'FOHISTX',
       'FODINNERX', 'FOLIBRAYX', 'FOBOOKSTX', 'FOCONCRTX', 'FOMUSEUMX',
       'FOZOOX', 'FOGROUPX', 'FOSPRTEVX ', 'HHENGLISH', 'CSPEAKX',
       'HHTOTALXX', 'HHPRTNRSX', 'P1REL', 'P1SEX', 'P1MRSTA ', 'P1AGE',
       'P2GUARD ', 'P2AGE', 'P2REL', 'P2SEX', 'P2MRSTA', 'PAR1EMPL',
       'PAR1FTFY', 'PAR2FTFY', 'NUMSIBSX', 'TTLHHINC', 'OWNRNTHB',
       'HVINTSPHO', 'HVINTCOM', 'INTACC', 'CHLDNT', 'LRNCOMP', 'LRNTAB',
       'LRNCELL'], dtype=object)

In [4]:
# Filter columns by new list
new_df = data.filter(col_list)
new_df.count()

SEGRADES      16446
CENREG        16446
EDCPUB        16446
EDCCAT        16446
EDCREL        16446
EDCPRI        16446
EDCINTK12     16446
EDCHSFL       16446
DISTASSI      16446
SCHRTSCHL     16446
SCHLMAGNET    16446
SEENJOY       16446
SEABSNT       16446
FCSCHOOL      16446
FCTEACHR      16446
FCSTDS        16446
FHHOME        16446
FHWKHRS       16446
FOSTORY2X     16446
FOCRAFTS      16446
FOGAMES       16446
FOBUILDX      16446
FOSPORT       16446
FORESPON      16446
FOHISTX       16446
FODINNERX     16446
FOLIBRAYX     16446
FOBOOKSTX     16446
FOCONCRTX     16446
FOMUSEUMX     16446
FOZOOX        16446
FOGROUPX      16446
HHENGLISH     16446
CSPEAKX       16446
HHTOTALXX     16446
HHPRTNRSX     16446
P1REL         16446
P1SEX         16446
P1AGE         16446
P2AGE         16446
P2REL         16446
P2SEX         16446
P2MRSTA       16446
PAR1EMPL      16446
PAR1FTFY      16446
PAR2FTFY      16446
NUMSIBSX      16446
TTLHHINC      16446
OWNRNTHB      16446
HVINTSPHO     16446


In [5]:
# Remove rows with -1 or -9 as a value
values = [-1, -9]
grades = [5]
columns = list(new_df)
print(columns)

for col in columns:
    if (col == "SEGRADES"):
        new_df = new_df[new_df[col].isin(grades) == False]
    else:
        new_df = new_df[new_df[col].isin(values) == False]
new_df

['SEGRADES', 'CENREG', 'EDCPUB', 'EDCCAT', 'EDCREL', 'EDCPRI', 'EDCINTK12', 'EDCHSFL', 'DISTASSI', 'SCHRTSCHL', 'SCHLMAGNET', 'SEENJOY', 'SEABSNT', 'FCSCHOOL', 'FCTEACHR', 'FCSTDS', 'FHHOME', 'FHWKHRS', 'FOSTORY2X', 'FOCRAFTS', 'FOGAMES', 'FOBUILDX', 'FOSPORT', 'FORESPON', 'FOHISTX', 'FODINNERX', 'FOLIBRAYX', 'FOBOOKSTX', 'FOCONCRTX', 'FOMUSEUMX', 'FOZOOX', 'FOGROUPX', 'HHENGLISH', 'CSPEAKX', 'HHTOTALXX', 'HHPRTNRSX', 'P1REL', 'P1SEX', 'P1AGE', 'P2AGE', 'P2REL', 'P2SEX', 'P2MRSTA', 'PAR1EMPL', 'PAR1FTFY', 'PAR2FTFY', 'NUMSIBSX', 'TTLHHINC', 'OWNRNTHB', 'HVINTSPHO', 'HVINTCOM', 'INTACC', 'CHLDNT', 'LRNCOMP', 'LRNTAB', 'LRNCELL']


,SEGRADES,CENREG,EDCPUB,EDCCAT,EDCREL,EDCPRI,EDCINTK12,EDCHSFL,DISTASSI,SCHRTSCHL,...,NUMSIBSX,TTLHHINC,OWNRNTHB,HVINTSPHO,HVINTCOM,INTACC,CHLDNT,LRNCOMP,LRNTAB,LRNCELL
0,1,4,1,2,2,2,2,2,1,2,...,1,8,2,1,1,1,1,1,2,1
4,1,2,1,2,2,2,2,2,1,2,...,1,10,1,1,1,1,2,1,2,2
7,1,2,1,2,2,2,2,2,1,2,...,2,3,1,1,1,1,1,1,1,1
9,1,3,1,2,2,2,2,2,2,2,...,1,11,1,1,1,1,1,1,1,2
11,2,3,1,2,2,2,2,2,2,2,...,2,3,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16438,1,1,1,2,2,2,2,2,2,2,...,1,8,1,1,1,1,1,1,1,1
16439,1,1,1,2,2,2,2,2,1,2,...,2,4,2,1,1,1,1,1,1,1
16441,1,3,1,2,2,2,2,2,1,2,...,2,9,1,1,1,1,2,1,1,1
16443,1,1,1,2,2,2,2,2,1,1,...,2,2,2,1,1,1,1,1,2,1


In [6]:
# Check for rows with -1 as a value
count = new_df[new_df.columns[1:]].eq(-1).sum().to_frame().T
for col, row in count.iteritems():
    print(col)
    print(row.values)

CENREG
[0]
EDCPUB
[0]
EDCCAT
[0]
EDCREL
[0]
EDCPRI
[0]
EDCINTK12
[0]
EDCHSFL
[0]
DISTASSI
[0]
SCHRTSCHL
[0]
SCHLMAGNET
[0]
SEENJOY
[0]
SEABSNT
[0]
FCSCHOOL
[0]
FCTEACHR
[0]
FCSTDS
[0]
FHHOME
[0]
FHWKHRS
[0]
FOSTORY2X
[0]
FOCRAFTS
[0]
FOGAMES
[0]
FOBUILDX
[0]
FOSPORT
[0]
FORESPON
[0]
FOHISTX
[0]
FODINNERX
[0]
FOLIBRAYX
[0]
FOBOOKSTX
[0]
FOCONCRTX
[0]
FOMUSEUMX
[0]
FOZOOX
[0]
FOGROUPX
[0]
HHENGLISH
[0]
CSPEAKX
[0]
HHTOTALXX
[0]
HHPRTNRSX
[0]
P1REL
[0]
P1SEX
[0]
P1AGE
[0]
P2AGE
[0]
P2REL
[0]
P2SEX
[0]
P2MRSTA
[0]
PAR1EMPL
[0]
PAR1FTFY
[0]
PAR2FTFY
[0]
NUMSIBSX
[0]
TTLHHINC
[0]
OWNRNTHB
[0]
HVINTSPHO
[0]
HVINTCOM
[0]
INTACC
[0]
CHLDNT
[0]
LRNCOMP
[0]
LRNTAB
[0]
LRNCELL
[0]


In [7]:
len(new_df)
new_df

,SEGRADES,CENREG,EDCPUB,EDCCAT,EDCREL,EDCPRI,EDCINTK12,EDCHSFL,DISTASSI,SCHRTSCHL,...,NUMSIBSX,TTLHHINC,OWNRNTHB,HVINTSPHO,HVINTCOM,INTACC,CHLDNT,LRNCOMP,LRNTAB,LRNCELL
0,1,4,1,2,2,2,2,2,1,2,...,1,8,2,1,1,1,1,1,2,1
4,1,2,1,2,2,2,2,2,1,2,...,1,10,1,1,1,1,2,1,2,2
7,1,2,1,2,2,2,2,2,1,2,...,2,3,1,1,1,1,1,1,1,1
9,1,3,1,2,2,2,2,2,2,2,...,1,11,1,1,1,1,1,1,1,2
11,2,3,1,2,2,2,2,2,2,2,...,2,3,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16438,1,1,1,2,2,2,2,2,2,2,...,1,8,1,1,1,1,1,1,1,1
16439,1,1,1,2,2,2,2,2,1,2,...,2,4,2,1,1,1,1,1,1,1
16441,1,3,1,2,2,2,2,2,1,2,...,2,9,1,1,1,1,2,1,1,1
16443,1,1,1,2,2,2,2,2,1,1,...,2,2,2,1,1,1,1,1,2,1


In [8]:
new_df.apply(lambda x: x.value_counts())

,SEGRADES,CENREG,EDCPUB,EDCCAT,EDCREL,EDCPRI,EDCINTK12,EDCHSFL,DISTASSI,SCHRTSCHL,...,NUMSIBSX,TTLHHINC,OWNRNTHB,HVINTSPHO,HVINTCOM,INTACC,CHLDNT,LRNCOMP,LRNTAB,LRNCELL
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2148.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5042.0,1374.0,8314.0,18.0,9.0,28.0,41.0,20.0,7204.0,665.0,...,3954.0,119.0,6913.0,8254.0,8105.0,8017.0,4974.0,7071.0,4853.0,5971.0
2,2564.0,3082.0,39.0,8335.0,8344.0,8325.0,8312.0,8333.0,1149.0,7688.0,...,1660.0,215.0,1352.0,99.0,248.0,88.0,2714.0,1282.0,3500.0,2382.0
3,649.0,1885.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,440.0,403.0,88.0,NaN,NaN,237.0,554.0,NaN,NaN,NaN
4,98.0,2012.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,110.0,484.0,NaN,NaN,NaN,11.0,111.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# drop school types since nearly all are public school (EDCPUB)
new_df = new_df.drop(['EDCPUB', 'EDCCAT', 'EDCREL', 'EDCPRI', 'EDCINTK12', 'EDCHSFL'], axis=1)

In [10]:
new_df.apply(lambda x: x.value_counts())

,SEGRADES,CENREG,DISTASSI,SCHRTSCHL,SCHLMAGNET,SEENJOY,SEABSNT,FCSCHOOL,FCTEACHR,FCSTDS,...,NUMSIBSX,TTLHHINC,OWNRNTHB,HVINTSPHO,HVINTCOM,INTACC,CHLDNT,LRNCOMP,LRNTAB,LRNCELL
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2148.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5042.0,1374.0,7204.0,665.0,543.0,2929.0,6944.0,5263.0,4945.0,5045.0,...,3954.0,119.0,6913.0,8254.0,8105.0,8017.0,4974.0,7071.0,4853.0,5971.0
2,2564.0,3082.0,1149.0,7688.0,7810.0,4477.0,1114.0,2629.0,2913.0,2772.0,...,1660.0,215.0,1352.0,99.0,248.0,88.0,2714.0,1282.0,3500.0,2382.0
3,649.0,1885.0,NaN,NaN,NaN,789.0,231.0,354.0,410.0,418.0,...,440.0,403.0,88.0,NaN,NaN,237.0,554.0,NaN,NaN,NaN
4,98.0,2012.0,NaN,NaN,NaN,158.0,64.0,107.0,85.0,118.0,...,110.0,484.0,NaN,NaN,NaN,11.0,111.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Consilidate grades into pass or fail
passing = [1]
failing = range(2 , 4, 1)

print(failing)

new_df["SEGRADES"] = [0 if x in failing else 1 for x in new_df["SEGRADES"]]

# for index, row in new_df.iterrows():
#     if row[1] == passing[0]:
#         row[1] = 1
#     if row[1] in failing:
#         print(row[1])
#         print("Found")
#         row[1] = "Find"
#         print(row[1])
        
# Consilidate P1AGE ahd P2AGE
range_1 = range(0, 20, 1)
range_2 = range(20, 40, 1)
range_3 = range(40, 60, 1)
range_4 = range(60, 80, 1)
range_5 = range(80, 100, 1)

# P1AGE
new_df["P1AGE"] = [(1 if x in range_1 else (2 if x in range_2 else (3 if x in range_3 else (4 if x in range_4 else 5)))) for x in new_df["P1AGE"]]

# P2AGE
new_df["P2AGE"] = [(1 if x in range_1 else (2 if x in range_2 else (3 if x in range_3 else (4 if x in range_4 else 5)))) for x in new_df["P2AGE"]]

# Consolidate FHWKHRS
range_1 = range(0, 20, 1)
range_2 = range(20, 40, 1)
range_3 = range(40, 60, 1)
range_4 = range(60, 80, 1)

new_df["FHWKHRS"] = [(1 if x in range_1 else (2 if x in range_2 else (3 if x in range_3 else 4))) for x in new_df["FHWKHRS"]]
        
# Consolidate FODINNERX
new_df["FODINNERX"] = [(1 if x <= 0 <= 2 else (2 if x <= 3 <= 4 else (3 if x <= 5 <= 6 else 4))) for x in new_df["FODINNERX"]]

new_df

range(2, 4)


,SEGRADES,CENREG,DISTASSI,SCHRTSCHL,SCHLMAGNET,SEENJOY,SEABSNT,FCSCHOOL,FCTEACHR,FCSTDS,...,NUMSIBSX,TTLHHINC,OWNRNTHB,HVINTSPHO,HVINTCOM,INTACC,CHLDNT,LRNCOMP,LRNTAB,LRNCELL
0,1,0,1,2,2,2,1,2,2,1,...,1,8,2,1,1,1,1,1,2,1
4,1,0,1,2,2,2,1,1,1,1,...,1,10,1,1,1,1,2,1,2,2
7,1,0,1,2,2,1,1,1,2,1,...,1,3,1,1,1,1,1,1,1,1
9,1,0,2,2,2,2,1,1,1,1,...,1,11,1,1,1,1,1,1,1,2
11,0,0,2,2,2,1,1,2,2,2,...,1,3,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16438,1,1,2,2,2,2,1,1,2,2,...,1,8,1,1,1,1,1,1,1,1
16439,1,1,1,2,2,3,1,2,1,1,...,1,4,2,1,1,1,1,1,1,1
16441,1,0,1,2,2,1,1,1,1,1,...,1,9,1,1,1,1,2,1,1,1
16443,1,1,1,1,2,2,1,1,1,1,...,1,2,2,1,1,1,1,1,2,1


In [16]:
kept_col = []
for col in new_df.columns:
    kept_col.append(col)
kept_col

['SEGRADES',
 'CENREG',
 'DISTASSI',
 'SCHRTSCHL',
 'SCHLMAGNET',
 'SEENJOY',
 'SEABSNT',
 'FCSCHOOL',
 'FCTEACHR',
 'FCSTDS',
 'FHHOME',
 'FHWKHRS',
 'FOSTORY2X',
 'FOCRAFTS',
 'FOGAMES',
 'FOBUILDX',
 'FOSPORT',
 'FORESPON',
 'FOHISTX',
 'FODINNERX',
 'FOLIBRAYX',
 'FOBOOKSTX',
 'FOCONCRTX',
 'FOMUSEUMX',
 'FOZOOX',
 'FOGROUPX',
 'HHENGLISH',
 'CSPEAKX',
 'HHTOTALXX',
 'HHPRTNRSX',
 'P1REL',
 'P1SEX',
 'P1AGE',
 'P2AGE',
 'P2REL',
 'P2SEX',
 'P2MRSTA',
 'PAR1EMPL',
 'PAR1FTFY',
 'PAR2FTFY',
 'NUMSIBSX',
 'TTLHHINC',
 'OWNRNTHB',
 'HVINTSPHO',
 'HVINTCOM',
 'INTACC',
 'CHLDNT',
 'LRNCOMP',
 'LRNTAB',
 'LRNCELL']

In [17]:
new_df

,SEGRADES,CENREG,DISTASSI,SCHRTSCHL,SCHLMAGNET,SEENJOY,SEABSNT,FCSCHOOL,FCTEACHR,FCSTDS,...,NUMSIBSX,TTLHHINC,OWNRNTHB,HVINTSPHO,HVINTCOM,INTACC,CHLDNT,LRNCOMP,LRNTAB,LRNCELL
0,1,0,1,2,2,2,1,2,2,1,...,1,8,2,1,1,1,1,1,2,1
4,1,0,1,2,2,2,1,1,1,1,...,1,10,1,1,1,1,2,1,2,2
7,1,0,1,2,2,1,1,1,2,1,...,1,3,1,1,1,1,1,1,1,1
9,1,0,2,2,2,2,1,1,1,1,...,1,11,1,1,1,1,1,1,1,2
11,0,0,2,2,2,1,1,2,2,2,...,1,3,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16438,1,1,2,2,2,2,1,1,2,2,...,1,8,1,1,1,1,1,1,1,1
16439,1,1,1,2,2,3,1,2,1,1,...,1,4,2,1,1,1,1,1,1,1
16441,1,0,1,2,2,1,1,1,1,1,...,1,9,1,1,1,1,2,1,1,1
16443,1,1,1,1,2,2,1,1,1,1,...,1,2,2,1,1,1,1,1,2,1


In [18]:
new_df.apply(lambda x: x.value_counts())

,SEGRADES,CENREG,DISTASSI,SCHRTSCHL,SCHLMAGNET,SEENJOY,SEABSNT,FCSCHOOL,FCTEACHR,FCSTDS,...,NUMSIBSX,TTLHHINC,OWNRNTHB,HVINTSPHO,HVINTCOM,INTACC,CHLDNT,LRNCOMP,LRNTAB,LRNCELL
0,3213.0,6979.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5140.0,1374.0,7204.0,665.0,543.0,2929.0,6944.0,5263.0,4945.0,5045.0,...,8353.0,119.0,6913.0,8254.0,8105.0,8017.0,4974.0,7071.0,4853.0,5971.0
2,NaN,NaN,1149.0,7688.0,7810.0,4477.0,1114.0,2629.0,2913.0,2772.0,...,NaN,215.0,1352.0,99.0,248.0,88.0,2714.0,1282.0,3500.0,2382.0
3,NaN,NaN,NaN,NaN,NaN,789.0,231.0,354.0,410.0,418.0,...,NaN,403.0,88.0,NaN,NaN,237.0,554.0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,158.0,64.0,107.0,85.0,118.0,...,NaN,484.0,NaN,NaN,NaN,11.0,111.0,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,488.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,496.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,745.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1335.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1855.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
binary_encoded = pd.get_dummies(new_df, columns=kept_col)

In [ ]:
binary_encoded.to_csv("../Resources/nhes_19_pfi_cleaned.csv")